In [1]:
from keras import layers
from keras import models
from keras import optimizers
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np
from numpy import random
import os
from os.path import isfile, join
import pandas as pd
from math import ceil
%matplotlib inline

Using TensorFlow backend.


In [35]:
class ConvnetSneakers:
    def __init__(self, base_image_path, master_image_size, master_color_channels):
        self.master_image_size = master_image_size
        self.base_image_path = base_image_path
        self.master_color_channels = master_color_channels
    
    def get_new_model(self):
        """
        Creates a new CNN model. It also prints a summary of the
        model.

        It uses the external variables master_image_size and
        master_color_channels to setup the input layer.

        Returns
        -------
        keras.models.Sequential
            The model ready to be trained
        """
        model = models.Sequential()
        model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(self.master_image_size[0],
                                                                            self.master_image_size[1],
                                                                            self.master_color_channels)))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(layers.MaxPooling2D((2, 2)))
        model.add(layers.Flatten())
        model.add(layers.Dense(256, activation='relu'))
        model.add(layers.Dense(1, activation='sigmoid'))
        model.compile(loss='binary_crossentropy', optimizer=optimizers.RMSprop(lr=0.5e-4), metrics=['acc'])

        model.summary()

        return model
    
    def train_validation_test(self, shuffle_seed=0):
        """
        Reads images in a directory and splits them up according to class

        Assumes a binary classification, with EQUAL COUNTS in each
        class.

        This automatically assigns classes to the images. It assumes
        the name of each class is the first token of the filename as
        delimited by "_".

        The dataframes returned are to be used by Keras. They have the
        columns "filename" and "class" to point to the images and the
        classes, repectively.

        Parameters
        ----------
        shuffle_seed : int
            The integer to seed the random number generator which shuffles
            the dataframe.

        Returns
        -------
        pd.DataFrame, pd.DataFrame, pd.DataFrame
            The train, validation and test sets, respectively.
        """
        print(f'>>> Shuffle seed {shuffle_seed}')

        train_fraction = 0.8
        random.seed(shuffle_seed)

        all_images_list = []

        for filename in os.listdir(self.base_image_path):
            if isfile(join(self.base_image_path, filename)):
                image_class = filename.split('.')[0]
                all_images_list.append({'class': image_class, 'filename': filename})

        all_images = pd.DataFrame(all_images_list)
        all_images = all_images.sample(frac=1).reset_index(drop=True)
        all_classes = all_images['class'].unique()

        first_class_name = all_classes[0]
        second_class_name = all_classes[1]

        first_class = all_images.copy().where(all_images['class'] == first_class_name).dropna()
        second_class = all_images.copy().where(all_images['class'] == second_class_name).dropna()

        train_row_count = int(len(first_class) * train_fraction)
        test_val_count = len(first_class) - train_row_count

        first_class_train = first_class.iloc[2 * test_val_count:]
        first_class_val = first_class.iloc[test_val_count:2 * test_val_count]
        first_class_test = first_class.iloc[0:test_val_count]

        second_class_train = second_class.iloc[2 * test_val_count:]
        second_class_val = second_class.iloc[test_val_count:2 * test_val_count]
        second_class_test = second_class.iloc[0:test_val_count]

        train = first_class_train.append(second_class_train).reset_index().drop('index', axis=1)
        val = first_class_val.append(second_class_val).reset_index().drop('index', axis=1)
        test = first_class_test.append(second_class_test).reset_index().drop('index', axis=1)

        print(first_class_name, second_class_name)

        return train, val, test
    
    def train_validation_test_generators(self, shuffle_seed=0):
        """
        Creates generators for train, validation and test datasets. These
        can then be used by Keras to train a model.

        Dataframe shuffling is prevented at this step because the dataframe
        is assumed to have been shuffled beforehand with an RNG with a known
        seed.

        The dataframe is created for you from the images in src_dir. See the
        train_validation_test function for more information.

        Parameters
        ----------
        shuffle_seed : int
            The seed for the RNG used for dataframe shuffling.

        Returns
        -------
        """
        train, validation, test = self.train_validation_test(shuffle_seed)

        train_datagen = image.ImageDataGenerator(rescale=1.0/255)
        test_datagen = image.ImageDataGenerator(rescale=1.0/255)
        validation_datagen = image.ImageDataGenerator(rescale=1.0/255)

        train_generator = train_datagen.flow_from_dataframe(dataframe=train,
                                                            directory=self.base_image_path,
                                                            target_size=self.master_image_size,
                                                            batch_size=20,
                                                            shuffle=False,
                                                            color_mode='grayscale',
                                                            class_mode='binary')

        validation_generator = train_datagen.flow_from_dataframe(dataframe=validation,
                                                          directory=self.base_image_path,
                                                          target_size=self.master_image_size,
                                                          batch_size=20,
                                                          shuffle=False,
                                                          color_mode='grayscale',
                                                          class_mode='binary')

        test_generator = train_datagen.flow_from_dataframe(dataframe=test,
                                                           directory=self.base_image_path,
                                                           target_size=self.master_image_size,
                                                           batch_size=20,
                                                           shuffle=False,
                                                           color_mode='grayscale',
                                                           class_mode='binary')

        return train_generator, validation_generator, test_generator
    
    def train_model_and_get_history(self, shuffle_seed=0, epochs=10):
        """
        Trains a model and returns the RNG seed and history

        Parameters
        ----------
        shuffle_seed : int
            The seed for the RNG that shuffles the train, validation
            and test datasets.

        Returns
        -------
        keras.model.Sequential, dict
            The model that created the history
        """
        train_generator, validation_generator, test_generator = self.train_validation_test_generators(shuffle_seed)
        model = self.get_new_model()
        train_history = model.fit_generator(train_generator,
                                            steps_per_epoch=100,
                                            epochs=epochs,
                                            validation_data=validation_generator,
                                            validation_steps=50,
                                            verbose=1)
        return model, train_history
    
    def train_and_run_different_shuffles(self, shuffles=10, epochs_per_shuffle=10):
        """
        This runs a sequence of trainings, each with a different shuffle
        controlled by a different random number generator.
        
        Parameters
        ----------
        shuffles : int
            The number of shuffles and training runs to go.
            
        epochs_per_shuffle : int
            The number of epochs in each shuffle of the data.
        """
        histories = []
        for shuffle_seed in range(shuffles):
            model, history = self.train_model_and_get_history(shuffle_seed, epochs=epochs_per_shuffle)
            train, validation, test = self.train_validation_test(shuffle_seed)
            histories.append({
                'shuffle_seed': shuffle_seed,
                'epochs': epochs_per_shuffle,
                'val_accs': history.history['val_acc'],
                'model': model,
                'train': train,
                'validation': validation,
                'test': test
            })
        return histories

In [36]:
trainer = ConvnetSneakers(master_image_size=(256, 256), base_image_path='grayscale-256x256', master_color_channels=1)
histories = trainer.train_and_run_different_shuffles(2, 2)

>>> Shuffle seed 0
benzene_ring non_benzene_ring
Found 246 validated image filenames belonging to 2 classes.
Found 82 validated image filenames belonging to 2 classes.
Found 82 validated image filenames belonging to 2 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_46 (Conv2D)           (None, 254, 254, 64)      640       
_________________________________________________________________
max_pooling2d_46 (MaxPooling (None, 127, 127, 64)      0         
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 125, 125, 64)      36928     
_________________________________________________________________
max_pooling2d_47 (MaxPooling (None, 62, 62, 64)        0         
_________________________________________________________________
conv2d_48 (Conv2D)           (None, 60, 60, 64)        36928     
_______________________________________________

In [37]:
histories

[{'shuffle_seed': 0,
  'epochs': 2,
  'val_accs': [0.5731707335245319, 0.5731707346148607],
  'model': <keras.engine.sequential.Sequential at 0x7f819aba2cf8>,
  'train':                 class                   filename
  0    non_benzene_ring  non_benzene_ring.0187.jpg
  1    non_benzene_ring  non_benzene_ring.0192.jpg
  2    non_benzene_ring  non_benzene_ring.9978.jpg
  3    non_benzene_ring  non_benzene_ring.0180.jpg
  4    non_benzene_ring  non_benzene_ring.0119.jpg
  5    non_benzene_ring  non_benzene_ring.0171.jpg
  6    non_benzene_ring  non_benzene_ring.0181.jpg
  7    non_benzene_ring  non_benzene_ring.0172.jpg
  8    non_benzene_ring  non_benzene_ring.0160.jpg
  9    non_benzene_ring  non_benzene_ring.0134.jpg
  10   non_benzene_ring  non_benzene_ring.9967.jpg
  11   non_benzene_ring  non_benzene_ring.9972.jpg
  12   non_benzene_ring  non_benzene_ring.0118.jpg
  13   non_benzene_ring  non_benzene_ring.0194.jpg
  14   non_benzene_ring  non_benzene_ring.0201.jpg
  15   non_benze